In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://es.wikipedia.org/wiki/Anexo:Pa%C3%ADses_de_Am%C3%A9rica_Latina_por_%C3%ADndice_de_desarrollo_humano'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
header = []
tbody = soup.findAll("tbody")[1]
for country in tbody.findAll('th'):
    header.append(country.string) #return tags
header = list(map(lambda x: x[0:-1],header))
header = header[0:-1]
header

['Pais', '2018', '2015', '2010', '2005', '2000', '1995', '1990', 'Cambio']

In [5]:
df = pd.DataFrame(columns=header[1:])

In [6]:
countries = []
tbody = soup.findAll("tbody")[1]
for country in tbody:
    try:
        if not country.td.find('a').get('title') in countries:
            countries.append(country.td.find('a').get('title'))
            
    except:
        continue
        

print(countries)

['Chile', 'Argentina', 'Uruguay', 'Panamá', 'Costa Rica', 'Cuba', 'México', 'Brasil', 'Colombia', 'República Dominicana', 'Venezuela', 'Paraguay', 'Bolivia', 'El Salvador', 'Guatemala', 'Nicaragua', 'Honduras', 'Haití']


In [7]:
idh_per_year = []
tbody = soup.findAll("tbody")[1]
country_idx = 0
for country in tbody:
    try:
        for column in range(1, len(header) + 1):
            value_per_year = country.findAll('td')[column]#index indicates years value
            for value in value_per_year:
                idh_per_year.append(value)
        

    except:
        continue
    ###start ad hoc clean stage###
    idh_per_year = list(map(lambda x: str(x), idh_per_year))
    idh_per_year = list(map(lambda x: x[0:-1], idh_per_year))
    idh_per_year = list(map(lambda x: x.replace(',','.'), idh_per_year))
    for idx, value in enumerate(idh_per_year):
        try:
            value = float(value)
            idh_per_year[idx] = value
        except:
            idh_per_year.pop(idx)
        
        if isinstance(value, str):
            try:
                idh_per_year.pop(idx)
            except:
                pass
        else:
            continue
    ###end up ad hoc clean stage###
    try:
        
        df.loc[countries[country_idx]] = idh_per_year
        idh_per_year.clear()
        country_idx += 1
    except:
        pass

In [8]:
df

,2018,2015,2010,2005,2000,1995,1990,Cambio
Chile,0.847,0.839,0.800,0.788,0.753,0.726,0.703,0.144
Argentina,0.830,0.828,0.818,0.777,0.770,0.731,0.707,0.123
Uruguay,0.808,0.802,0.774,0.756,0.742,0.710,0.692,0.116
Panamá,0.795,0.782,0.758,0.739,0.719,0.688,0.659,0.136
Costa Rica,0.794,0.786,0.754,0.727,0.711,0.686,0.655,0.139
Cuba,0.779,0.768,0.777,0.730,0.686,0.654,0.676,0.102
México,0.767,0.759,0.739,0.729,0.705,0.672,0.652,0.115
Brasil,0.760,0.755,0.726,0.699,0.684,0.650,0.613,0.148
Colombia,0.761,0.753,0.729,0.692,0.662,0.633,0.599,0.162
República Dominicana,0.745,0.733,0.700,0.675,0.653,0.624,0.593,0.152


In [9]:
df.to_csv('idh_latin_america.csv')